In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta

# --- CONFIGURATION (Remains Same) ---
ASSETS = {
    'EURUSD': 'EURUSD=X', 'USDCHF': 'USDCHF=X', 'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X', 'BTCUSD': 'BTC-USD', 'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X', 'XAGUSD': 'XAGUSD=X', 'SP500m': '^GSPC', 'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'), 'D1': ('1d', '5y'), '4H': ('4h', '2y'),
    '1H': ('1h', '2y'), 'M30': ('30m', '60d'), 'M15': ('15m', '60d')
}

def flatten_yf_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df

# def detect_fvg_signals(data, lookback_period=14, body_multiplier=1.5, box_width=20):
#     fvg_list = [None] * len(data)
#     time_col = 'Datetime' if 'Datetime' in data.columns else 'Date'

#     for i in range(2, len(data)):
#         # 1. Detect FVG (Your Strict Logic)
#         first_high, first_low = data['High'].iloc[i-2], data['Low'].iloc[i-2]
#         mid_open, mid_close = data['Open'].iloc[i-1], data['Close'].iloc[i-1]
#         third_low, third_high = data['Low'].iloc[i], data['High'].iloc[i]

#         mid_body = abs(mid_close - mid_open)
#         avg_body = abs(data['Close'] - data['Open']).iloc[max(0, i-1-lookback_period):i-1].mean()
#         min_gap = mid_body * 0.10
#         fvg_timestamp = data[time_col].iloc[i-1]

#         fvg_type, bottom, top = None, 0, 0
#         if third_low > first_high:
#             if (mid_body > avg_body * body_multiplier) and (third_low - first_high > min_gap):
#                 fvg_type, bottom, top = 'bullish', first_high, third_low
#         elif third_high < first_low:
#             if (mid_body > avg_body * body_multiplier) and (first_low - third_high > min_gap):
#                 fvg_type, bottom, top = 'bearish', first_low, third_high

#         if fvg_type:
#             # 2. Track for Retracement + Continuation Signal
#             signal_time = None
#             is_mitigated = False
            
#             check_range = min(i + box_width, len(data))
#             for j in range(i + 1, check_range):
#                 curr_high, curr_low = data['High'].iloc[j], data['Low'].iloc[j]
#                 curr_close, curr_open = data['Close'].iloc[j], data['Open'].iloc[j]

#                 if fvg_type == 'bullish':
#                     # Pullback check: Price touches or enters the gap
#                     if curr_low <= top:
#                         is_mitigated = True
#                         # Signal: Price respects the gap (Close stays above bottom) and candle is Bullish
#                         if curr_close > bottom and curr_close > curr_open:
#                             signal_time = data[time_col].iloc[j]
#                             break
#                         # Failure: If price closes below the gap, orderflow is broken
#                         if curr_close < bottom: break 
#                 else: # Bearish
#                     if curr_high >= bottom:
#                         is_mitigated = True
#                         if curr_close < top and curr_close < curr_open:
#                             signal_time = data[time_col].iloc[j]
#                             break
#                         if curr_close > top: break

#             fvg_list[i] = {
#                 'type': fvg_type, 'bottom': bottom, 'top': top, 
#                 'start': fvg_timestamp, 'idx': i, 
#                 'signal_time': signal_time, 'mitigated': is_mitigated
#             }
#     return fvg_list


def detect_fvg_signals(data, lookback_period=14, body_multiplier=1.5, box_width=25):
    fvg_list = [None] * len(data)
    time_col = 'Datetime' if 'Datetime' in data.columns else 'Date'

    for i in range(2, len(data)):
        # 1. CORE FVG DETECTION
        first_high, first_low = data['High'].iloc[i-2], data['Low'].iloc[i-2]
        mid_open, mid_close = data['Open'].iloc[i-1], data['Close'].iloc[i-1]
        third_low, third_high = data['Low'].iloc[i], data['High'].iloc[i]

        mid_body = abs(mid_close - mid_open)
        avg_body = abs(data['Close'] - data['Open']
                       ).iloc[max(0, i-1-lookback_period):i-1].mean()
        min_gap = mid_body * 0.10
        fvg_timestamp = data[time_col].iloc[i-1]

        fvg_type, bottom, top = None, 0, 0
        if third_low > first_high:
            if (mid_body > avg_body * body_multiplier) and (third_low - first_high > min_gap):
                fvg_type, bottom, top = 'bullish', first_high, third_low
        elif third_high < first_low:
            if (mid_body > avg_body * body_multiplier) and (first_low - third_high > min_gap):
                fvg_type, bottom, top = 'bearish', first_low, third_high

        if fvg_type:
            signal_time = None
            is_mitigated = False

            # 2. SEARCH FOR PULLBACK & 3/5 MOMENTUM SIGNAL
            # Ensure space for 5-candle check
            check_range = min(i + box_width, len(data) - 5)
            for j in range(i + 1, check_range):
                curr_high, curr_low = data['High'].iloc[j], data['Low'].iloc[j]

                # Condition: Price touches the zone
                touched = (fvg_type == 'bullish' and curr_low <= top) or \
                          (fvg_type == 'bearish' and curr_high >= bottom)

                if touched:
                    is_mitigated = True

                    # --- NEW LOGIC: Check next 5 candles for 3 directional closes ---
                    directional_count = 0
                    for k in range(j, min(j + 5, len(data))):
                        c_open, c_close = data['Open'].iloc[k], data['Close'].iloc[k]

                        if fvg_type == 'bullish' and c_close > c_open:  # Green candle
                            directional_count += 1
                        elif fvg_type == 'bearish' and c_close < c_open:  # Red candle
                            directional_count += 1

                    # If 3 or more out of 5 are directional, we have a signal
                    if directional_count >= 3:
                        signal_time = data[time_col].iloc[j]
                        break

                    # Orderflow Protection: Stop if price closes through the box
                    if fvg_type == 'bullish' and data['Close'].iloc[j] < bottom:
                        break
                    if fvg_type == 'bearish' and data['Close'].iloc[j] > top:
                        break

            fvg_list[i] = {
                'type': fvg_type, 'bottom': bottom, 'top': top,
                'start': fvg_timestamp, 'idx': i,
                'signal_time': signal_time, 'mitigated': is_mitigated
            }
    return fvg_list
def plot_fvg_signals(pair, timeframe, candle_limit=100, box_width=20):
    interval, period = TIMEFRAME_SETTINGS[timeframe]
    df = yf.download(ASSETS[pair], period=period, interval=interval, auto_adjust=False, progress=False)
    if df.empty: return
    df = flatten_yf_columns(df).reset_index()
    time_col = 'Datetime' if 'Datetime' in df.columns else 'Date'

    df['FVG_RESULTS'] = detect_fvg_signals(df, box_width=box_width)
    df_plot = df.tail(candle_limit).copy()

    fig = go.Figure(data=[go.Candlestick(
        x=df_plot[time_col], open=df_plot["Open"], high=df_plot["High"], 
        low=df_plot["Low"], close=df_plot["Close"], name=pair
    )])

    for _, row in df_plot.iterrows():
        res = row['FVG_RESULTS']
        if res:
            end_idx = min(res['idx'] + box_width, len(df) - 1)
            end_time = df[time_col].iloc[end_idx]
            
            # Color logic: Differentiate mitigated (pullback) vs fresh zones
            color = "rgba(0, 255, 150, 0.3)" if res['type'] == 'bullish' else "rgba(255, 50, 50, 0.3)"
            if res['mitigated']:
                color = "rgba(200, 200, 200, 0.2)" # Neutralize if already touched

            fig.add_shape(type="rect", x0=res['start'], x1=end_time, y0=res['bottom'], y1=res['top'],
                          fillcolor=color, layer="below", line=dict(width=0))

            # PLOT THE CONTINUATION SIGNAL
            if res['signal_time'] and res['signal_time'] in df_plot[time_col].values:
                fig.add_trace(go.Scatter(
                    x=[res['signal_time']], 
                    y=[res['top'] if res['type'] == 'bullish' else res['bottom']],
                    mode="markers",
                    marker=dict(symbol="star", size=12, color="yellow", line=dict(width=1, color="orange")),
                    name="Continuation Signal"
                ))

    fig.update_layout(title=f"<b>{pair} {timeframe}</b> - FVG Orderflow Rejection", 
                      template="plotly_dark", xaxis_rangeslider_visible=False, width=1200, height=750)
    fig.show()

if __name__ == "__main__":
    plot_fvg_signals('EURUSD', 'M30', candle_limit=120, box_width=30)